# Load fit results from A1 and A2

In [1]:
using JLD
files = readdir("saves")

all_phi1_full_orig = Array(Any, 100)
all_phi1_full_log = Array(Any, 100)
all_phi1_red_orig = Array(Any, 100)
all_phi1_red_log = Array(Any, 100)

phi0_red_orig = []
phi0_red_log = []
phi0_full = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];

n = 100
for i = 1:n
    # 10% errors
    phi1_red_orig, phi1_red_log, fname = load("saves/figure_A2_hess_"*string(i)*".jld", "phi1_orig", "phi1_log", "A1_filename")
    
    phi1_full_orig, phi1_full_log = load("saves/"*fname, "phi1_orig", "phi1_log")
    
    
    if i == 1
        phi0_red_orig, phi0_red_log = load("saves/figure_A2_hess_"*string(i)*".jld", "phi0_orig", "phi0_log")
    end
        
    all_phi1_full_orig[i] = phi1_full_orig
    all_phi1_full_log[i] = phi1_full_log
    all_phi1_red_orig[i] = phi1_red_orig
    all_phi1_red_log[i] = phi1_red_log
end

In [2]:
all_phi1_full_orig = hcat(all_phi1_full_orig[1:n]...)
all_phi1_full_log = hcat(all_phi1_full_log[1:n]...)
all_phi1_red_orig = hcat(all_phi1_red_orig[1:n]...)
all_phi1_red_log = hcat(all_phi1_red_log[1:n]...)


6x100 Array{Float64,2}:
 1.08516e-7  1.14341e-7  8.35819e-8  …  1.34475e-7  1.07503e-7  1.26801e-7
 6.85043     6.74525     7.4412         6.38284     6.89248     6.53743   
 0.541606    0.539264    0.534699       0.546425    0.542109    0.543052  
 3.64174e-5  3.58142e-5  3.67141e-5     3.5696e-5   3.76775e-5  3.68202e-5
 4.59533     4.34331     4.56318        4.54282     4.61157     4.46872   
 2.33752     2.46561     2.32594     …  2.3794      2.35647     2.44121   

In [ ]:
tmp = load("saves/figure_A1_hess_"*"8"*".jld") # Example with diverging parameters on fit
#tmp = load("saves/figure_A1_hess_"*"96"*".jld") # Example with diverging parameters on fit


# Plot things here

using NBInclude
nbinclude("MBAM_plots.ipynb")
import MBAM_plots

include("../BK_functions/bk_setup_script.jl")


PlotlyJS.plot(MBAM_plots.plot(reshape(BK_simulator(phi0, hcat(x_grid...))[:], 8, 13)'))

PlotlyJS.plot(MBAM_plots.plot(reshape(tmp["ydata_orig"], 8, 13)'))

PlotlyJS.plot(MBAM_plots.plot(reshape(BK_simulator(tmp["phi1_orig"], hcat(x_grid...))[:], 8, 13)'))

[phi0 tmp["phi1_orig"]]

PlotlyJS.plot([MBAM_plots.plot(reshape(BK_simulator(phi0, hcat(x_grid...))[:], 8, 13)', mode="markers", color=0) 
                MBAM_plots.plot(reshape(BK_simulator(tmp["phi1_orig"], hcat(x_grid...))[:], 8, 13)', mode="lines")
    ])

# Generate CSV files for plotting elsewhere

In [ ]:
# Output the simulated data point markers in a CSV format with: 
# first column - voltage, second column - Popen, third column error top, fourth column

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for original model
    tmp_vals = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp_vals.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp_vals[:], errbar[:], errbar[:]))
end

# Also output the example fit with just first col - voltage, second col - Popen
# but with voltage sampled more densely to make it a continous line

Vdense = minimum(V):0.001:maximum(V) # voltage sampled every 1 millivolt

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    println(ca_str)
    
    # Generate grid to evaluate over
    x_grid_cur = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    
    
    # Get the function values for the fit to noisy data
    tmp_vals = BK_simulator(tmp["phi1_orig"], hcat(x_grid_cur...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp_vals[:]))
end

In [ ]:
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 tmp["phi1_orig"]])

In [ ]:
# Output the statistics from all our data:
# For each model and assay
# and for each parameter 
# we will look at the standard deviation of the order of magnitude of fitted values: std(log10(fit_vals))

# First column: Original model Original Assay
# Second column: Reduced model Original Assay
# Third column: Original model Log Assay
# Fourth column: Reduced model Log Assay

writecsv("CSV/figure_A2_panel1_orig_orig.csv", std(log10(all_phi1_full_orig) ,2))
writecsv("CSV/figure_A2_panel1_red_orig.csv", std(log10(all_phi1_red_orig) ,2))
writecsv("CSV/figure_A2_panel1_orig_log.csv", std(log10(all_phi1_full_log) ,2))
writecsv("CSV/figure_A2_panel1_red_log.csv", std(log10(all_phi1_red_log) ,2))